In [1]:
import time
from itertools import compress
from skimage.util.shape import view_as_windows
import matplotlib.pyplot as plt
import numpy as np  
import os

Concatenate files into one big file first

In [2]:
def concatenate_and_save(folder_path, output_directory, output_file):
    files = os.listdir(folder_path)
    print(files)

    np_files = [f for f in files if f.endswith('.npy')]
    arrays = [np.load(os.path.join(folder_path, f)) for f in np_files]
    concatenated_array = np.concatenate(arrays, axis=0)
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    output_path = os.path.join(output_directory, output_file)
    print("Stacked array shape:", concatenated_array.shape)
    
    np.save(output_path, concatenated_array)
    print("Concatenation complete. Array saved as", output_path)

In [49]:
path = "data_download/seis_train"
output_file = "seistrain_full.npy"
concatenate_and_save(path, path, output_file)

['seistrain1.npy', 'seistrain2.npy', 'seistrain3.npy', 'seistrain4.npy', 'seistrain5.npy', 'seistrain6.npy', 'seistrain7.npy', 'seistrain8.npy', 'seistrain9.npy']
Stacked array shape: (900, 3174, 1537)
Concatenation complete. Array saved as data_download/seis_train\seistrain_full.npy


In [56]:
path = "data_download/seis_val"
output_file = "seisval_full.npy"
concatenate_and_save(path, path, output_file)

['seisval1.npy', 'seisval2.npy']
Stacked array shape: (200, 3174, 1537)
Concatenation complete. Array saved as data_download/seis_val\seisval_full.npy


In [57]:
path = "data_download/seis_test"
output_file = "seistest_full.npy"
concatenate_and_save(path, path, output_file)

['seistest1.npy', 'seistest2.npy', 'seistest3.npy', 'seistest4.npy', 'seistest5.npy', 'seistest6.npy', 'seistest7.npy']
Stacked array shape: (703, 3174, 1537)
Concatenation complete. Array saved as data_download/seis_test\seistest_full.npy


### Load the training dataset, apply normalization and prepare patches

Since 32.7GB memory is needed for seistrain dataset normalization. thus it is applied before the stacking of individual seistrain file 

In [3]:
def prepare_norm(input_path, output_folder):
    seismicfiles = os.listdir(input_path)
    print(seismicfiles)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in seismicfiles:
        print(file_name)
        path = os.path.join(input_path+file_name)
        seismic = np.load(path)
        seismic = np.moveaxis(seismic,-2,-1)         # re-sort in the order of # IL, Z, XL  
        seismic = (seismic-seismic.min(axis=(1,2), keepdims=True))/(seismic.max(axis=(1,2), keepdims=True)-seismic.min(axis=(1,2), keepdims=True))
        output_path = os.path.join(output_folder, file_name[:-4] + '_norm.npy')
        np.save(output_path, seismic)
        print("Process complete. Array saved as", output_path)
    

In [78]:
path = "data_download/seis_train/"
output_path = "data_download/full_data/norm"
prepare_norm(path,output_path)

['seistrain1.npy', 'seistrain2.npy', 'seistrain3.npy', 'seistrain4.npy', 'seistrain5.npy', 'seistrain6.npy', 'seistrain7.npy', 'seistrain8.npy', 'seistrain9.npy']
seistrain1.npy
Process complete. Array saved as data_download/full_data/norm\seistrain1_norm.npy
seistrain2.npy
Process complete. Array saved as data_download/full_data/norm\seistrain2_norm.npy
seistrain3.npy
Process complete. Array saved as data_download/full_data/norm\seistrain3_norm.npy
seistrain4.npy
Process complete. Array saved as data_download/full_data/norm\seistrain4_norm.npy
seistrain5.npy
Process complete. Array saved as data_download/full_data/norm\seistrain5_norm.npy
seistrain6.npy
Process complete. Array saved as data_download/full_data/norm\seistrain6_norm.npy
seistrain7.npy
Process complete. Array saved as data_download/full_data/norm\seistrain7_norm.npy
seistrain8.npy
Process complete. Array saved as data_download/full_data/norm\seistrain8_norm.npy
seistrain9.npy
Process complete. Array saved as data_download

In [5]:
path = "data_download/full_data/norm/"
output_path = "data_download/full_data/"
output_file = "seistrain_norm_full.npy"
concatenate_and_save(path, output_path, output_file)

['seistrain1_norm.npy', 'seistrain2_norm.npy', 'seistrain3_norm.npy', 'seistrain4_norm.npy', 'seistrain5_norm.npy', 'seistrain6_norm.npy', 'seistrain7_norm.npy', 'seistrain8_norm.npy', 'seistrain9_norm.npy']
Stacked array shape: (900, 1537, 3174)
Concatenation complete. Array saved as data_download/full_data/seistrain_norm_full.npy


In [2]:
path = "data_download/full_data/"
seistrain = np.load(os.path.join(path,"seistrain_norm_full.npy"))
faulttrain = np.load(os.path.join(path,"faulttrain_full.npy"))

In [9]:
faulttrain= np.moveaxis(faulttrain,-2,-1) # IL, Z, XL

In [19]:
print('Seismic train shape',seistrain.shape)
print('Fault train shape',faulttrain.shape)

Seismic train shape (900, 1537, 3174)
Fault train shape (900, 1537, 3174)


In [9]:
print(seistrain.max(),seistrain.min(), faulttrain.max(), faulttrain.min())

1.0 0.0 True False


setup for the splitting patches

In [11]:
IL, Z, XL = faulttrain.shape

im_height = Z
im_width = XL
splitsize = 96
stepsize = 48
overlapsize = splitsize-stepsize
pixelThre = int(0.03*splitsize*splitsize)
print(pixelThre)

276


In [12]:
horizontal_splits_number = int(np.ceil((im_width-overlapsize)/stepsize))
print("horizontal_splits_number", horizontal_splits_number)
width_after_pad = stepsize*horizontal_splits_number+overlapsize
print("width_after_pad", width_after_pad)
left_pad = int((width_after_pad-im_width)/2)
right_pad = width_after_pad-im_width-left_pad
print("left_pad,right_pad",left_pad,right_pad)

vertical_splits_number = int(np.ceil((im_height-overlapsize)/stepsize))
print("vertical_splits_number",vertical_splits_number)
height_after_pad = stepsize*vertical_splits_number+overlapsize
print("height_after_pad",height_after_pad)
top_pad = int((height_after_pad-im_height)/2)
bottom_pad = height_after_pad-im_height-top_pad
print("top_pad,bottom_pad", top_pad,bottom_pad)

horizontal_splits_number 66
width_after_pad 3216
left_pad,right_pad 21 21
vertical_splits_number 32
height_after_pad 1584
top_pad,bottom_pad 23 24


In [13]:
def split_Image(bigImage,isMask,top_pad,bottom_pad,left_pad,right_pad,splitsize,stepsize,vertical_splits_number,horizontal_splits_number):
#     print(bigImage.shape)
    if isMask==True:
        arr = np.pad(bigImage,((top_pad,bottom_pad),(left_pad,right_pad)),"reflect")
        splits = view_as_windows(arr, (splitsize,splitsize),step=stepsize)
        splits = splits.reshape((vertical_splits_number*horizontal_splits_number,splitsize,splitsize))
    else: 
        arr = np.pad(bigImage,((top_pad,bottom_pad),(left_pad,right_pad),(0,0)),"reflect")
        splits = view_as_windows(arr, (splitsize,splitsize,3),step=stepsize)
        splits = splits.reshape((vertical_splits_number*horizontal_splits_number,splitsize,splitsize,3))
    return splits # return list of arrays.

In [13]:
t_start = time.time()
X = []
Y = []
for i in range(0,900,1):
    mask = faulttrain[i]
    splits = split_Image(mask, True,top_pad,bottom_pad,left_pad,right_pad,splitsize,stepsize,vertical_splits_number,horizontal_splits_number)
#     print(splits.shape)
    t = (splits.sum((1,2)) < pixelThre)
    no_label_element_index = list(compress(range(len(t)), t))
    # get all the indexes of the no label pieces by adding elements in axis 2 and 3.
    splits = np.delete(splits, no_label_element_index,0) # delete element i along axis 0
#     print("splits.shape", splits.shape)
    Y.extend(splits)
    
    img = seistrain[i]
    splits = split_Image(img, True,top_pad,bottom_pad,left_pad,right_pad,splitsize,stepsize,vertical_splits_number,horizontal_splits_number)
    splits = np.delete(splits, no_label_element_index,0) # delete element i along axis 0
#     print("splits.shape", splits.shape)
    X.extend(splits)
#     break

print(len(Y))
print(len(X))
print(X[0].shape)
print("read images in {} sec".format(time.time()-t_start))

181029
181029
(96, 96)
read images in 318.36458945274353 sec


In [14]:
print(X[0].dtype)
print(Y[0].dtype)
X = np.asarray(X, dtype=np.float32)
Y = np.asarray(Y, dtype=np.float32)
print(X[0].dtype)
print(Y[0].dtype)

float64
bool
float32
float32


In [16]:
directory = "data_download/full_data/patches/seismic/train"
if not os.path.exists(directory):
    os.makedirs(directory)
directory = "data_download/full_data/patches/fault/train"
if not os.path.exists(directory):
    os.makedirs(directory)

In [17]:
patches_path = 'data_download/full_data/patches'

for i in range(len(X)):
    np.save("{}/seismic/train/{}.npy".format(patches_path, i),X[i])
    np.save("{}/fault/train/{}.npy".format(patches_path, i),Y[i])

### Load the validation dataset, apply normalization and prepare patches

In [3]:
path = "data_download/full_data/"
seisval = np.load(os.path.join(path,"seisval_full.npy"))
faultval = np.load(os.path.join(path,"faultval_full.npy"))

In [8]:
seisval= np.moveaxis(seisval,-2,-1) 
faultval= np.moveaxis(faultval,-2,-1)

In [9]:
print('Seismic validation shape',seisval.shape)
print('Fault validation shape',faultval.shape)

Seismic validation shape (200, 1537, 3174)
Fault validation shape (200, 1537, 3174)


In [10]:
seisval = (seisval-seisval.min(axis=(1,2), keepdims=True))/(seisval.max(axis=(1,2), keepdims=True)-seisval.min(axis=(1,2), keepdims=True))

In [11]:
print(seisval.max(),seisval.min(), faultval.max(), faultval.min())

1.0 0.0 True False


In [23]:
t_start = time.time()
X = []
Y = []
for i in range(0,200,1):
    mask = faultval[i]
    splits = split_Image(mask, True,top_pad,bottom_pad,left_pad,right_pad,splitsize,stepsize,vertical_splits_number,horizontal_splits_number)
#     print(splits.shape)
    t = (splits.sum((1,2)) < pixelThre)
    no_label_element_index = list(compress(range(len(t)), t))
    # get all the indexes of the no label pieces by adding elements in axis 2 and 3.
    splits = np.delete(splits, no_label_element_index,0) # delete element i along axis 0
#     print("splits.shape", splits.shape)
    Y.extend(splits)
    
    img = seisval[i]
    splits = split_Image(img, True,top_pad,bottom_pad,left_pad,right_pad,splitsize,stepsize,vertical_splits_number,horizontal_splits_number)
    splits = np.delete(splits, no_label_element_index,0) # delete element i along axis 0
    X.extend(splits)

print(len(Y))
print(len(X))
print(X[0].shape)
print("read images in {} sec".format(time.time()-t_start))

64317
64317
(96, 96)
read images in 101.6322500705719 sec


In [24]:
X = np.asarray(X, dtype=np.float32)
Y = np.asarray(Y, dtype=np.float32)

In [25]:
directory = "data_download/full_data/patches/seismic/val"
if not os.path.exists(directory):
    os.makedirs(directory)
directory = "data_download/full_data/patches/fault/val"
if not os.path.exists(directory):
    os.makedirs(directory)

In [26]:
patches_path = 'data_download/full_data/patches'

for i in range(len(X)):
    np.save("{}/seismic/val/{}.npy".format(patches_path, i),X[i])
    np.save("{}/fault/val/{}.npy".format(patches_path, i),Y[i])

### the same procedure for test dataset

In [4]:
path = "data_download/seis_test/"
output_path = "data_download/full_data/seistest_norm"
prepare_norm(path,output_path)

['seistest1.npy', 'seistest2.npy', 'seistest3.npy', 'seistest4.npy', 'seistest5.npy', 'seistest6.npy', 'seistest7.npy']
seistest1.npy
Process complete. Array saved as data_download/full_data/seistest_norm\seistest1_norm.npy
seistest2.npy
Process complete. Array saved as data_download/full_data/seistest_norm\seistest2_norm.npy
seistest3.npy
Process complete. Array saved as data_download/full_data/seistest_norm\seistest3_norm.npy
seistest4.npy
Process complete. Array saved as data_download/full_data/seistest_norm\seistest4_norm.npy
seistest5.npy
Process complete. Array saved as data_download/full_data/seistest_norm\seistest5_norm.npy
seistest6.npy
Process complete. Array saved as data_download/full_data/seistest_norm\seistest6_norm.npy
seistest7.npy
Process complete. Array saved as data_download/full_data/seistest_norm\seistest7_norm.npy


In [3]:
path = "data_download/full_data/seistest_norm/"
output_path = "data_download/full_data/"
output_file = "seistest_norm_full.npy"
concatenate_and_save(path, output_path, output_file)

['seistest1_norm.npy', 'seistest2_norm.npy', 'seistest3_norm.npy', 'seistest4_norm.npy', 'seistest5_norm.npy', 'seistest6_norm.npy', 'seistest7_norm.npy']
Stacked array shape: (703, 1537, 3174)
Concatenation complete. Array saved as data_download/full_data/seistest_norm_full.npy


In [4]:
path = "data_download/full_data/"
seistest = np.load(os.path.join(path,"seistest_norm_full.npy"))
faulttest = np.load(os.path.join(path,"faulttest_full.npy"))

In [5]:
faulttest= np.moveaxis(faulttest,-2,-1)

In [6]:
print('Seismic test shape',seistest.shape)
print('Fault test shape',faulttest.shape)

Seismic test shape (703, 1537, 3174)
Fault test shape (703, 1537, 3174)


In [7]:
print(seistest.max(),seistest.min(), faulttest.max(), faulttest.min())

1.0 0.0 True False


In [14]:
t_start = time.time()
X = []
Y = []
for i in range(0,200,1):
    mask = faulttest[i]
    splits = split_Image(mask, True,top_pad,bottom_pad,left_pad,right_pad,splitsize,stepsize,vertical_splits_number,horizontal_splits_number)
#     print(splits.shape)
    t = (splits.sum((1,2)) < pixelThre)
    no_label_element_index = list(compress(range(len(t)), t))
    # get all the indexes of the no label pieces by adding elements in axis 2 and 3.
    splits = np.delete(splits, no_label_element_index,0) # delete element i along axis 0
#     print("splits.shape", splits.shape)
    Y.extend(splits)
    
    img = seistest[i]
    splits = split_Image(img, True,top_pad,bottom_pad,left_pad,right_pad,splitsize,stepsize,vertical_splits_number,horizontal_splits_number)
    splits = np.delete(splits, no_label_element_index,0) # delete element i along axis 0
    X.extend(splits)

print(len(Y))
print(len(X))
print(X[0].shape)
print("read images in {} sec".format(time.time()-t_start))

66077
66077
(96, 96)
read images in 101.14579248428345 sec


In [15]:
X = np.asarray(X, dtype=np.float32)
Y = np.asarray(Y, dtype=np.float32)

In [16]:
directory = "data_download/full_data/patches/seismic/test"
if not os.path.exists(directory):
    os.makedirs(directory)
directory = "data_download/full_data/patches/fault/test"
if not os.path.exists(directory):
    os.makedirs(directory)

In [17]:
patches_path = 'data_download/full_data/patches'

for i in range(len(X)):
    np.save("{}/seismic/test/{}.npy".format(patches_path, i),X[i])
    np.save("{}/fault/test/{}.npy".format(patches_path, i),Y[i])